In [1]:
%load_ext autoreload
%load_ext autotime

time: 407 µs (started: 2023-03-12 14:59:50 -07:00)


In [2]:
%run init.ipynb

time: 2.24 s (started: 2023-03-12 14:59:50 -07:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from wisio.utils.file_utils import ensure_dir
from wisio.utils.json_encoders import NpEncoder
from wisio.utils.logger import create_logger, format_log

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 600 ms (started: 2023-03-12 14:59:52 -07:00)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.65.157:8788/status,
Dashboard: http://192.168.65.157:8788/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.65.157:39657,Workers: 0
Dashboard: http://192.168.65.157:8788/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 66.4 ms (started: 2023-03-12 14:59:53 -07:00)


In [5]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 917 µs (started: 2023-03-12 14:59:53 -07:00)


In [7]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/lbann-cosmoflow/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=2218,,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 6.62 s (started: 2023-03-12 14:59:58 -07:00)


In [8]:
ddf.head()

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,proc_name,file_name,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,1,0,2099194032,2,0,0.295049,3.254283,MPI_Comm_dup,0,...,app1,UNKNOWN,UNKNOWN,0,0,0.0,2.959234,17746659,0,0
1,2,1,0,2099194032,0,3,3.253641,3.253651,open,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000010,32536459,3341432783455051907,-929352282003167793
2,3,1,0,2099194032,0,3,3.253686,3.253690,open,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000004,32536878,3341432783455051907,-929352282003167793
3,4,1,0,2099194032,0,3,3.253701,3.253709,ftruncate,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000008,32537048,3341432783455051907,-929352282003167793
4,5,1,0,2099194032,0,3,3.253717,3.253719,close,1,...,app1,app1#localhost#0#35186471282864,/dev/shm/job2154496201-35619-OMPI_COLL_IBM-0-c...,0,0,0.0,0.000002,32537178,3341432783455051907,-929352282003167793


time: 2min 24s (started: 2023-03-12 15:00:05 -07:00)


In [9]:
ddf.index.count().compute()

2225473791

time: 1min 5s (started: 2023-03-12 12:06:54 -07:00)


In [18]:
ddf['tend'].max().compute()

12023159.0

time: 13.4 s (started: 2023-03-11 22:04:29 -08:00)


In [19]:
ddf['file_id'].unique().count().compute()

52779

time: 10 s (started: 2023-03-11 22:04:43 -08:00)


In [20]:
ddf['proc_id'].unique().count().compute()

693

time: 10.4 s (started: 2023-03-11 22:04:53 -08:00)


In [9]:
global_min_max = load_global_min_max(log_dir=log_dir)
global_min_max

{'file_id': [-7860256618999200354, 9202064004080513284],
 'proc_id': [-929352282003204857, -929352281994824808],
 'tmid': [0, 120231585016530]}

time: 4.2 ms (started: 2023-03-12 15:02:29 -07:00)


In [10]:
from dask.distributed import Future, get_client
from typing import Union

DELTA_BINS = [
    0,
    0.001,
    0.01,
    0.1,
    0.25,
    0.5,
    0.75,
    1
]
DELTA_BIN_LABELS = [
    'none',
    'trivial',
    'very low',
    'low',
    'medium',
    'high',
    'very high',
    'critical'
]
TIME_PRECISION = 1e7

def set_metric_deltas(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, csp_col, delta_col = (
        f"{metric}_sum",
        f"{metric}_csp",
        f"{metric}_delta",
    )
    df[csp_col] = df[metric_col].cumsum() / max_io_time
    df[delta_col] = df[csp_col].diff().fillna(df[csp_col])
    return df


def set_metric_percentages(df: pd.DataFrame, metric: str, max_io_time: float):
    metric_col, pero_col, perr_col = (
        f"{metric}_sum",
        f"{metric}_pero",
        f"{metric}_perr"
    )
    df[pero_col] = df[metric_col] / max_io_time
    df[perr_col] = df[metric_col] / df[metric_col].sum()
    return df


def set_metric_scores(df: pd.DataFrame, metric: str, col: str):
    bin_col, score_col, threshold_col = (
        f"{metric}_bin",
        f"{metric}_score",
        f"{metric}_th",
    )
    df[bin_col] = np.digitize(df[col], bins=DELTA_BINS, right=True)
    df[score_col] = np.choose(df[bin_col] - 1, choices=DELTA_BIN_LABELS, mode='clip')
    df[threshold_col] = np.choose(df[bin_col] - 1, choices=DELTA_BINS, mode='clip')
    return df.drop(columns=[bin_col])

def compute_tranges(global_min_max: dict, precision=TIME_PRECISION):
    tmid_min, tmid_max = global_min_max['tmid']
    tranges = np.arange(tmid_min, tmid_max, precision)
    return get_client().scatter(tranges)


def set_tranges(df: pd.DataFrame, tranges: Union[Future, np.ndarray]):
    tranges = tranges.result() if isinstance(tranges, Future) else tranges
    return df.assign(trange=np.digitize(df['tmid'], bins=tranges, right=True))

time: 4.33 ms (started: 2023-03-12 15:02:29 -07:00)


In [11]:
tranges = compute_tranges(global_min_max=global_min_max)
tranges

<Future: finished, type: numpy.ndarray, key: ndarray-266d6a682b6af71a63655b2416c915b7>

time: 1.1 s (started: 2023-03-12 15:02:29 -07:00)


In [13]:
VIEW_TYPES = ['trange', 'file_name', 'proc_name']

time: 719 µs (started: 2023-03-12 15:11:02 -07:00)


In [14]:
from copy import copy

view_types = copy(VIEW_TYPES)
view_types.append('io_cat')
view_types.append('acc_pat')
view_types.append('func_id')

main_view = ddf[(ddf['cat'] == 0) & (ddf['io_cat'].isin([1,2,3]))]  \
    .map_partitions(set_tranges, tranges=tranges) \
    .groupby(view_types) \
    .agg({
        'duration': [sum],
        'index': ['count'],
        'size': [min, max, sum],
#         'file_name': ['first'],
#         'proc_name': ['first'],
#         'func_id': [unique_flatten()],
    }) \
    .persist()

main_view


Dask DataFrame Structure:
              duration  index   size              
                   sum  count    min    max    sum
npartitions=1                                     
               float32  int64  int64  int64  int64
                   ...    ...    ...    ...    ...
Dask Name: aggregate-agg, 1 tasks

time: 1.14 s (started: 2023-03-12 15:11:02 -07:00)


In [15]:
wait(main_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-c36341e6614b96e6b326fe90a46b59af', 0)>}, not_done=set())

time: 1min 55s (started: 2023-03-12 15:11:04 -07:00)


In [16]:
main_view = main_view.reset_index()
main_view.columns = ['_'.join(tup).rstrip('_') for tup in main_view.columns.values]
main_view

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
npartitions=1,,,,,,,,,,,
,int64,object,object,int64,int64,object,float32,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...


time: 54.9 ms (started: 2023-03-12 15:13:00 -07:00)


In [18]:
wait(main_view)


DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('aggregate-agg-c36341e6614b96e6b326fe90a46b59af', 0)>}, not_done=set())

time: 5.09 ms (started: 2023-03-12 15:13:49 -07:00)


In [19]:
!rm -rf {log_dir}/cached & mkdir -p {log_dir}/cached

time: 143 ms (started: 2023-03-12 15:13:51 -07:00)


In [20]:
main_view.repartition('100MB').to_parquet(f"{log_dir}/cached/main_view")

[None]

time: 5.43 s (started: 2023-03-12 15:13:55 -07:00)


In [21]:
!ls -lh {log_dir}/cached/main_view | head -n 10

total 521M
-rw------- 1 izzet iopp 2.0K Mar 12 15:14 _common_metadata
-rw------- 1 izzet iopp  15K Mar 12 15:14 _metadata
-rw------- 1 izzet iopp  51M Mar 12 15:13 part.0.parquet
-rw------- 1 izzet iopp  53M Mar 12 15:13 part.1.parquet
-rw------- 1 izzet iopp  52M Mar 12 15:13 part.2.parquet
-rw------- 1 izzet iopp  50M Mar 12 15:13 part.3.parquet
-rw------- 1 izzet iopp  53M Mar 12 15:13 part.4.parquet
-rw------- 1 izzet iopp  53M Mar 12 15:13 part.5.parquet
-rw------- 1 izzet iopp  53M Mar 12 15:13 part.6.parquet
time: 6.6 s (started: 2023-03-12 15:14:11 -07:00)


In [22]:
del main_view

time: 719 µs (started: 2023-03-12 15:14:32 -07:00)


In [23]:
main_view = dd.read_parquet(f"{log_dir}/cached/main_view/*.parquet")
main_view

,trange,file_name,proc_name,io_cat,acc_pat,func_id,duration_sum,index_count,size_min,size_max,size_sum
npartitions=10,,,,,,,,,,,
0,int64,object,object,int64,int64,object,float32,int64,int64,int64,int64
263223,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
2369015,...,...,...,...,...,...,...,...,...,...,...
2632238,...,...,...,...,...,...,...,...,...,...,...


time: 88.3 ms (started: 2023-03-12 15:14:36 -07:00)


In [24]:
main_view['read_count'] = 0
main_view['write_count'] = 0
main_view['metadata_count'] = 0
main_view['read_count'] = main_view['read_count'].mask(main_view['io_cat'] == 1, main_view['index_count'])
main_view['write_count'] = main_view['write_count'].mask(main_view['io_cat'] == 2,  main_view['index_count'])
main_view['metadata_count'] = main_view['metadata_count'].mask(main_view['io_cat'] == 3,  main_view['index_count'])

main_view['read_size'] = 0
main_view['write_size'] = 0
main_view['metadata_size'] = 0
main_view['read_size'] = main_view['read_size'].mask(main_view['io_cat'] == 1, main_view['size_sum'])
main_view['write_size'] = main_view['write_size'].mask(main_view['io_cat'] == 2,  main_view['size_sum'])
main_view['metadata_size'] = main_view['metadata_size'].mask(main_view['io_cat'] == 3,  main_view['size_sum'])

main_view['read_time'] = 0
main_view['write_time'] = 0
main_view['metadata_time'] = 0
main_view['read_time'] = main_view['read_time'].mask(main_view['io_cat'] == 1, main_view['duration_sum'])
main_view['write_time'] = main_view['write_time'].mask(main_view['io_cat'] == 2,  main_view['duration_sum'])
main_view['metadata_time'] = main_view['metadata_time'].mask(main_view['io_cat'] == 3,  main_view['duration_sum'])

main_view['sequential_count'] = 0
main_view['random_count'] = 0
main_view['sequential_count'] = main_view['sequential_count'].mask(main_view['acc_pat'] == 0, main_view['index_count'])
main_view['random_count'] = main_view['random_count'].mask(main_view['acc_pat'] == 1, main_view['index_count'])

main_view['sequential_size'] = 0
main_view['random_size'] = 0
main_view['sequential_size'] = main_view['sequential_size'].mask(main_view['acc_pat'] == 0, main_view['size_sum'])
main_view['random_size'] = main_view['random_size'].mask(main_view['acc_pat'] == 1, main_view['size_sum'])

main_view['sequential_time'] = 0
main_view['random_time'] = 0
main_view['sequential_time'] = main_view['sequential_time'].mask(main_view['acc_pat'] == 0, main_view['duration_sum'])
main_view['random_time'] = main_view['random_time'].mask(main_view['acc_pat'] == 1, main_view['duration_sum'])


main_view['data_count'] = main_view['write_count'] + main_view['read_count']
main_view['data_size'] = main_view['write_size'] + main_view['read_size']
main_view['data_time'] = main_view['write_time'] + main_view['read_time']

main_view['open_time'] = 0
main_view['close_time'] = 0
main_view['seek_time'] = 0
main_view['stat_time'] = 0
main_view['open_time'] = main_view['open_time'].mask(main_view['func_id'].str.contains('open') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['close_time'] = main_view['close_time'].mask(main_view['func_id'].str.contains('close') & ~main_view['func_id'].str.contains('dir'),  main_view['duration_sum'])
main_view['seek_time'] = main_view['seek_time'].mask(main_view['func_id'].str.contains('seek'),  main_view['duration_sum'])
main_view['stat_time'] = main_view['stat_time'].mask(main_view['func_id'].str.contains('stat'),  main_view['duration_sum'])

# main_view['workload_type'] = 0
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['read_count'] == 0, 3)
# main_view['workload_type'] = main_view['workload_type'].mask(main_view['write_count'] == 0, 1)

# agg_dict = {col: sum for col in main_view.columns}
# agg_dict['file_name_first'] = 'first'
# agg_dict['proc_name_first'] = 'first'
# agg_dict['func_id_unique_flatten'] = unique_flatten()
# for view_type in VIEW_TYPES:
#     agg_dict.pop(view_type)

final_view = main_view \
    .groupby(VIEW_TYPES) \
    .sum() \
    .persist()

final_view


,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,write_time,metadata_time,sequential_count,random_count,sequential_size,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,float32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 787 ms (started: 2023-03-12 15:14:52 -07:00)


In [25]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-7b7266d6d88e305150aadc087af9dbdd', 0)>}, not_done=set())

time: 4.21 s (started: 2023-03-12 15:14:53 -07:00)


In [26]:
final_view = final_view.reset_index().persist()

time: 30.1 ms (started: 2023-03-12 15:15:38 -07:00)


In [27]:
wait(final_view)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('reset_index-dd7c1ec7961440075ba8265a4587990c', 0)>}, not_done=set())

time: 4.38 ms (started: 2023-03-12 15:15:39 -07:00)


In [29]:
final_view.repartition('100MB').to_parquet(f"{log_dir}/cached/final_view")

[None]

time: 1.74 s (started: 2023-03-12 15:16:01 -07:00)


In [30]:
!ls -lh {log_dir}/cached/final_view | head -n 10

total 299M
-rw------- 1 izzet iopp 4.9K Mar 12 15:16 _common_metadata
-rw------- 1 izzet iopp  18K Mar 12 15:16 _metadata
-rw------- 1 izzet iopp  74M Mar 12 15:16 part.0.parquet
-rw------- 1 izzet iopp  74M Mar 12 15:16 part.1.parquet
-rw------- 1 izzet iopp  76M Mar 12 15:16 part.2.parquet
-rw------- 1 izzet iopp  76M Mar 12 15:16 part.3.parquet
time: 608 ms (started: 2023-03-12 15:16:08 -07:00)


In [31]:
del final_view

time: 1.22 ms (started: 2023-03-12 15:16:20 -07:00)


In [32]:
final_view = dd.read_parquet(f"{log_dir}/cached/final_view/*.parquet")
final_view

,trange,file_name,proc_name,io_cat,acc_pat,duration_sum,index_count,size_min,size_max,size_sum,read_count,write_count,metadata_count,read_size,write_size,metadata_size,read_time,write_time,metadata_time,sequential_count,random_count,sequential_size,random_size,sequential_time,random_time,data_count,data_size,data_time,open_time,close_time,seek_time,stat_time
npartitions=4,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,object,object,int64,int64,float32,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64
207266,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414533,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621800,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
829066,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 120 ms (started: 2023-03-12 15:16:26 -07:00)


In [33]:
max_io_time = final_view.groupby(['proc_name']).agg({'duration_sum':sum})['duration_sum'].max()
max_io_time

dd.Scalar<series-..., dtype=float32>

time: 24 ms (started: 2023-03-12 15:16:41 -07:00)


In [34]:
DELTA = 0.0001
CUT = 0.5

time: 754 µs (started: 2023-03-12 15:16:41 -07:00)


In [35]:
import itertools as it

def _view_permutations(r: int):
    return it.permutations(VIEW_TYPES, r + 1)

metric = 'duration'
views = {}
for view_permutation in it.chain.from_iterable(map(_view_permutations, range(len(VIEW_TYPES)))):
    # Read types
    parent_type = view_permutation[:-1]
    view_type = view_permutation[-1]
    # Get parent view
    parent_view = views[parent_type] if parent_type in views else final_view
    # Create colum names
    metric_col, delta_col = f"{metric}_sum", f"{metric}_delta"
    # Check view type
    if view_type is not 'proc_name':
        # Compute `proc_id` view first
        group_view = parent_view \
            .groupby([view_type, 'proc_name']) \
            .agg({metric_col: sum}) \
            .groupby([view_type]) \
            .max()
    else:
        # Compute group view
        group_view = parent_view \
            .groupby([view_type]) \
            .agg({metric_col: sum})
    # Set metric scores
    group_view = group_view \
        .map_partitions(set_metric_deltas, metric=metric, max_io_time=max_io_time) \
        .query(f"{delta_col} > @delta", local_dict={'delta': DELTA})
    # Find filtered records
    view = parent_view.query(f"{view_type} in @indices", local_dict={'indices': group_view.index.unique()})
    # Return views
    views[view_permutation] = view

len(views)

15

time: 861 ms (started: 2023-03-12 15:16:50 -07:00)


In [36]:
for view_key, view in views.items():
    print(view_key, len(view))

('trange',) 505751
('file_name',) 773944
('proc_name',) 811835
('trange', 'file_name') 465197
('trange', 'proc_name') 500759
('file_name', 'trange') 464241
('file_name', 'proc_name') 749168
('proc_name', 'trange') 501404
('proc_name', 'file_name') 756912
('trange', 'file_name', 'proc_name') 460239
('trange', 'proc_name', 'file_name') 460239
('file_name', 'trange', 'proc_name') 459291
('file_name', 'proc_name', 'trange') 459299
('proc_name', 'trange', 'file_name') 460884
('proc_name', 'file_name', 'trange') 459863
time: 18.9 s (started: 2023-03-12 15:16:52 -07:00)


In [37]:
import dask.dataframe as dd
import pandas as pd
from dask import compute, delayed
from dask.distributed import get_client
from typing import Any, Dict

BOTTLENECK_ORDER = dict(
    file_name=('file_name', 'proc_name', 'trange'),
    proc_name=('proc_name', 'trange', 'file_name'),
    trange=('trange', 'proc_name', 'file_name'),
)
BOTTLENECK_TYPE = dict(
    file_name='file_name',
    proc_name='proc_name',
    trange='trange'
)

def _calculate_llc(level_row: pd.Series):
    llc = dict(level_row)
    for view_type in VIEW_TYPES:
        if view_type in llc:
            llc.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in llc:
            llc.pop(bottleneck_type)
    return llc


@delayed
def _process_bottleneck_view(
    view_key: tuple,
    threshold: float,
    low_level_view: pd.DataFrame,
    mid_level_view: pd.DataFrame,
    high_level_view: pd.DataFrame,
):
    # Get view type
    view_type = view_key[-1]
    # Get ordered bottleneck columns
    _, ml_col, ll_col = BOTTLENECK_ORDER[view_type]
    # Init bottlenecks
    bottlenecks = {}
    # Loop through index tuples
    ids_tuple = low_level_view.index
    for hl_id, ml_id, ll_id in ids_tuple:
        hl_row = high_level_view.loc[hl_id]
        ml_row = mid_level_view.loc[(hl_id, ml_id)]
        ll_row = low_level_view.loc[(hl_id, ml_id, ll_id)]
        if hl_id not in bottlenecks:
            bottlenecks[hl_id] = {}
            bottlenecks[hl_id]['llc'] = _calculate_llc(hl_row)
            bottlenecks[hl_id][ml_col] = {}
        if ml_id not in bottlenecks[hl_id][ml_col]:
            bottlenecks[hl_id][ml_col][ml_id] = {}
            bottlenecks[hl_id][ml_col][ml_id]['llc'] = _calculate_llc(ml_row)
            bottlenecks[hl_id][ml_col][ml_id][ll_col] = {}
        if ll_id not in bottlenecks[hl_id][ml_col][ml_id][ll_col]:
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id] = {}
            bottlenecks[hl_id][ml_col][ml_id][ll_col][ll_id]['llc'] = _calculate_llc(ll_row)
    # Return view key & bottlenecks
    return view_key, threshold, bottlenecks

def detect_bottlenecks(views: Dict[tuple, dd.DataFrame], view_types: list) -> Dict[tuple, object]:
    # Keep bottleneck views
    bottleneck_views = {}
    # Run through views
    for view_key, view in views.items():
        # Generate bottleneck views
        bottleneck_views[view_key] = _generate_bottlenecks_views(
            view_key=view_key,
            view=view,
            view_types=view_types,
            max_io_time=max_io_time,
            metric=metric,
        )
    # Generate bottlenecks
    bottlenecks = _process_bottleneck_views(
        bottleneck_views=bottleneck_views,
        metric=metric,
    )
    # Return bottleneck views
    return bottlenecks, bottleneck_views

def _process_bottleneck_views(bottleneck_views: Dict[tuple, dd.DataFrame], metric: str):
    # Init bottlenecks
    bottlenecks = {}
    bottlenecks_delayed = []
    # Run through bottleneck views
    for view_key, view_dict in bottleneck_views.items():
        # For given thresholds
        for th in DELTA_BINS[1:-1]:  # [0.25, 0.5, 0.75]

            threshold_col = f"{metric}_th"

            low_level_view = view_dict['low_level_view']
            mid_level_view = view_dict['mid_level_view']
            high_level_view = view_dict['high_level_view']

            bottlenecks_delayed.append(_process_bottleneck_view(
                view_key=view_key,
                threshold=th,
                low_level_view=low_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                mid_level_view=mid_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th}),
                high_level_view=high_level_view.query(f"{threshold_col} >= @th", local_dict={'th': th})
            ))

    # Compute all bottlenecks
    results = compute(*bottlenecks_delayed)
    for view_key, th, result in results:
        bottlenecks[view_key] = bottlenecks[view_key] if view_key in bottlenecks else {}
        bottlenecks[view_key][f"{th:.3f}"] = result
    # Return all bottlenecks
    return bottlenecks

def _generate_bottlenecks_views(
    view_key: tuple,
    view: dd.DataFrame,
    view_types: list,
    max_io_time: dd.core.Scalar,
    metric: str,
):
    # Get view type
    view_type = view_key[-1]
    bottleneck_type = BOTTLENECK_TYPE[view_type]

    # Create lower level view
    low_level_view = view \
        .groupby(list(BOTTLENECK_ORDER[view_type])) \
        .first() \
        .drop(columns=['acc_pat', 'io_cat', 'file_id', 'proc_id'], errors='ignore')

    # Non-proc agg columns
    non_proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=False)
    proc_agg_dict = _get_agg_dict(view_types=view_types, view_columns=low_level_view.columns, is_proc=True)

    # Create mid and high level views
    if bottleneck_type is not 'proc_name':
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'proc_name']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(proc_agg_dict)
    else:
        mid_level_view = low_level_view \
            .reset_index() \
            .groupby([bottleneck_type, 'trange']) \
            .agg(non_proc_agg_dict)

        high_level_view = mid_level_view \
            .reset_index() \
            .groupby([bottleneck_type]) \
            .agg(non_proc_agg_dict)

    low_level_view = low_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    mid_level_view = mid_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    high_level_view = high_level_view \
        .map_partitions(set_metric_percentages, metric=metric, max_io_time=max_io_time) \
        .map_partitions(set_metric_scores, metric=metric, col=f"{metric}_pero")

    return dict(
        low_level_view=low_level_view,
        mid_level_view=mid_level_view,
        high_level_view=high_level_view
    )

def _get_agg_dict(view_types: list, view_columns: list, is_proc=False):
    if is_proc:
        agg_dict = {col: max if any(x in col for x in 'duration time'.split()) else sum for col in view_columns}
    else:
        agg_dict = {col: sum for col in view_columns}
#     agg_dict['func_id'] = unique_flatten()
    agg_dict['size_min'] = min
    agg_dict['size_max'] = max
    for view_type in view_types:
        if view_type in agg_dict:
            agg_dict.pop(view_type)
        bottleneck_type = BOTTLENECK_TYPE[view_type]
        if bottleneck_type in agg_dict:
            agg_dict.pop(bottleneck_type)
    return agg_dict


time: 11.3 ms (started: 2023-03-12 15:17:14 -07:00)


In [38]:
bottlenecks, bottleneck_views = detect_bottlenecks(
    views=views,
    view_types=VIEW_TYPES,
)

len(bottlenecks)

15

time: 24.5 s (started: 2023-03-12 15:17:23 -07:00)


In [39]:
for view_key, bottleneck in bottlenecks.items():
    for th in bottleneck.keys():
        print(view_key, th, len(bottleneck[th]))

('trange',) 0.001 632
('trange',) 0.010 29
('trange',) 0.100 0
('trange',) 0.250 0
('trange',) 0.500 0
('trange',) 0.750 0
('file_name',) 0.001 325
('file_name',) 0.010 8
('file_name',) 0.100 0
('file_name',) 0.250 0
('file_name',) 0.500 0
('file_name',) 0.750 0
('proc_name',) 0.001 128
('proc_name',) 0.010 29
('proc_name',) 0.100 0
('proc_name',) 0.250 0
('proc_name',) 0.500 0
('proc_name',) 0.750 0
('trange', 'file_name') 0.001 325
('trange', 'file_name') 0.010 8
('trange', 'file_name') 0.100 0
('trange', 'file_name') 0.250 0
('trange', 'file_name') 0.500 0
('trange', 'file_name') 0.750 0
('trange', 'proc_name') 0.001 128
('trange', 'proc_name') 0.010 29
('trange', 'proc_name') 0.100 0
('trange', 'proc_name') 0.250 0
('trange', 'proc_name') 0.500 0
('trange', 'proc_name') 0.750 0
('file_name', 'trange') 0.001 632
('file_name', 'trange') 0.010 29
('file_name', 'trange') 0.100 0
('file_name', 'trange') 0.250 0
('file_name', 'trange') 0.500 0
('file_name', 'trange') 0.750 0
('file_name'

In [40]:
max_io_time.compute()

134.04991

time: 240 ms (started: 2023-03-12 15:18:29 -07:00)


In [41]:
!mkdir -p {log_dir}/bottlenecks

time: 154 ms (started: 2023-03-12 15:18:43 -07:00)


In [42]:
def save_bottlenecks(log_dir: str, bottlenecks: Dict[tuple, object]):
    bottleneck_dir = f"{log_dir}/bottlenecks"
    for view_key, bottleneck_dict in bottlenecks.items():
        file_name = '_'.join(view_key) if isinstance(view_key, tuple) else view_key
        with open(f"{bottleneck_dir}/{file_name}.json", 'w') as json_file:
            json.dump(bottleneck_dict, json_file, cls=NpEncoder, sort_keys=True)

time: 1.48 ms (started: 2023-03-12 15:18:50 -07:00)


In [43]:
save_bottlenecks(log_dir=log_dir, bottlenecks=bottlenecks)

time: 11.4 s (started: 2023-03-12 15:18:56 -07:00)
